In [3]:
!pip install torchaudio==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [4]:
!pip install pydub

In [5]:
import torchaudio
from tqdm import tqdm

In [6]:
print(torchaudio.__version__)

2.3.0+cu121


In [7]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os
import shutil
import json

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!unzip -qq "/content/drive/MyDrive/구음장애 dataset/original/dataset.zip" -d "/content/data"

In [16]:
root_path = "/content"
src_dir = "src_dir"
target_dir = "output_folder3"

In [12]:
import re

In [17]:
def remove_silence_dynamic_thresh(audio_file_name, src_dir, target_dir, root_path):

    print(f"audio_file_name: {audio_file_name} 무음 구간 제거 중   ...................")

    # src_audio_path = os.path.join(".", src_dir)
    src_audio_path = os.path.join(root_path, src_dir, audio_file_name)

    print("src_audio_path: ", src_audio_path)
    # audio_file_name = os.path.basename(src_audio_path)
    # tgt_audio_path = os.path.join(tgt_audio_dir, audio_file_name)

    # if os.path.exists(tgt_audio_path):
    #     print(f"\n{tgt_audio_path} already exists")
    #     return


    # 출력 파일명 설정 및 출력 폴더 생성
    print("target_dir: ", target_dir)
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        os.chdir(target_dir)
        os.makedirs("audio")
        os.makedirs("label")

    # os.chdir("..")
    # print("Done.................")
    # return

    # 오디오 파일 로드
    audio = AudioSegment.from_wav(src_audio_path)

    # 오디오의 평균 볼륨(dBFS) 계산
    average_loudness = audio.dBFS

    # silence_thresh를 평균 볼륨보다 조금 낮은 값으로 설정 (-10dB ~ -15dB 정도 낮추는 것이 일반적)
    silence_thresh = average_loudness - 10

    print(f"\nSource Audio File : {os.path.basename(src_audio_path)}",
          f"Average loudness: {average_loudness} dBFS",
          f"Silence threshold: {silence_thresh} dBFS"
    )



    # 무음 구간 제거
    chunks = split_on_silence(audio,
                              min_silence_len=2000,       # 1초 이상 무음을 감지
                              silence_thresh=silence_thresh, # 동적으로 설정된 무음 임계값
                              keep_silence=1000)  #축소된 무음 길이의 절반

    # # 무음이 제거된 오디오 합치기
    # processed_audio = AudioSegment.empty()
    # for chunk in chunks:
    #     processed_audio += chunk


    print(f"파일: {src_audio_path} 단위 파일 개수: {len(chunks)}   ...................")


    # 분리된 오디오 조각들 중 최대 15초까지만 선택하여 wav 파일로 저장
    result_audio_path = os.path.join(root_path, target_dir, "audio")
    result_label_path = os.path.join(root_path, target_dir, "label")

    chunk_name = audio_file_name[:-4]
    chunk_name_2 = ''
    for i, chunk in enumerate(chunks):
        # if len(chunk) > 15000:
        #    chunk = chunk[:15000]
        chunk_name_2 = f"{chunk_name}_{i:03d}.wav"
        # print(chunk_name_2)
        output_file = os.path.join(result_audio_path, chunk_name_2)
        chunk.export(output_file, format='wav')


    src_label_path = src_audio_path.replace('.wav', '.json')
    with open(src_label_path, 'r', encoding='utf-8') as f:
        text = json.load(f)
        transcript = text["Transcript"]
        # print(transcript)

        if '.' not in transcript:
            results = transcript.split(' ')
        else:
            transcript = re.sub(r'[^가-힣0-9.!?\s]', '', transcript) #텍스트 안에 한글,숫자, 공백, 문장부호만 남게!
            results = re.split('(?<=[.!?])', transcript) #텍스트 구분 문장부호로 하고 문장보호 보존!

        str_arr = [str for str in results]

        chunk_name = audio_file_name[:-4]
        target_label_path = os.path.join(root_path, target_dir, "label", chunk_name)
        # target_label_path = f"{target_dir}/label/{chunk_name}"
        for idx, str in enumerate(str_arr):
            if str: #빈 공백 텍스트 생성되지 않게!
                result_path = f"{target_label_path}_{idx:03d}.txt"
                # print(result_path)
                with open(result_path, 'w') as t:
                    t.write(str)


In [18]:
import os

os.chdir(root_path)


# # 파일 용량 threshold
audio_file_mb_thresh = 100

for item in tqdm(os.listdir(src_dir)):
    os.chdir(root_path)
    print("item: ", item)
    if item.endswith(".wav"):
        file_path = os.path.join(src_dir, item)
        if os.path.getsize(file_path) / (1024 * 1024) < audio_file_mb_thresh:
            # remove_silence_dynamic_thresh(file_path, tgt_audio_dir, tgt_label_dir)
            remove_silence_dynamic_thresh(item, src_dir, target_dir, root_path)


  0%|          | 0/3 [00:00<?, ?it/s]

item:  .ipynb_checkpoints
item:  ID-02-25-N-KSM-02-02-M-45-JL.wav
audio_file_name: ID-02-25-N-KSM-02-02-M-45-JL.wav 무음 구간 제거 중   ...................
src_audio_path:  /content/src_dir/ID-02-25-N-KSM-02-02-M-45-JL.wav
target_dir:  output_folder3

Source Audio File : ID-02-25-N-KSM-02-02-M-45-JL.wav Average loudness: -38.224477637504954 dBFS Silence threshold: -48.224477637504954 dBFS


100%|██████████| 3/3 [01:56<00:00, 38.68s/it]

파일: /content/src_dir/ID-02-25-N-KSM-02-02-M-45-JL.wav 단위 파일 개수: 60   ...................
item:  ID-02-25-N-KSM-02-02-M-45-JL.json
